### Analysis part 1: interest rates

In [45]:
import pandas as pd
import numpy as np
import sklearn as sk

from sklearn.model_selection import train_test_split

In [70]:
df = pd.read_csv('data/cleaned data/total_market_data_merged.csv')
#df = df.drop(['Unnamed: 0', "YEAR_x", "YEAR_y", 'DATE'], axis = 1)

In [47]:
df.head()

,Unnamed: 0,Five-Digit ZIP Code,year,Annual Change (%),HPI,HPI from 2012,HPI with 2012 base,normalized_sale_price,RECESSION_FLAG,avg_rate_for_year,Median_hh_income,Year_Avg_Unempl,state,zip_code,bucketed_year,State_Num,Resident Population,Percent Change in Resident Population
0,0,1001.0,1984,.,100.00,279.95,0.357207,58369.688570,0,10.225000,55828,7.7,Massachusetts,1001,1990,6.0,6016425.0,4.9
1,1,1001.0,1985,16.00,116.00,279.95,0.414360,67708.838742,0,8.100833,56871,7.2,Massachusetts,1001,1990,6.0,6016425.0,4.9
2,2,1001.0,1986,14.21,132.48,279.95,0.473227,77328.163418,0,6.805000,58920,6.7,Massachusetts,1001,1990,6.0,6016425.0,4.9
3,3,1001.0,1987,21.08,160.41,279.95,0.572995,93630.817436,0,6.657500,59624,6.2,Massachusetts,1001,1990,6.0,6016425.0,4.9
4,4,1001.0,1988,17.63,188.68,279.95,0.673977,110131.928394,0,7.568333,60115,5.5,Massachusetts,1001,1990,6.0,6016425.0,4.9


In [48]:
# get a sample from my zip code to later test
my_zip = df[(df['zip_code']==97405) & (df['year']==2022)]
my_zip

,Unnamed: 0,Five-Digit ZIP Code,year,Annual Change (%),HPI,HPI from 2012,HPI with 2012 base,normalized_sale_price,RECESSION_FLAG,avg_rate_for_year,Median_hh_income,Year_Avg_Unempl,state,zip_code,bucketed_year,State_Num,Resident Population,Percent Change in Resident Population
168440,168440,97405.0,2022,17.29,1245.59,604.3,2.061211,532739.0,0,1.683333,71000,3.8,Oregon,97405,2020,33.0,4237256.0,10.6


### Part 4/5/bonus: machine learning

In [83]:
# This is the data I'm going to make an annual change % with: zip code, year, recession flag, avg rate, state, population, pop change
new_data = np.array([[97405, 2023, 0, 5.0, 71000, 5, 33, 10.6],
                     [97405, 2024, 0, 5.0, 71000, 5, 33, 10.6],
                     [97405, 2025, 0, 5.0, 71000, 5, 33, 10.6]])

In [69]:
columns_with_nan = df.columns[df.isna().any()].tolist()
print(columns_with_nan)

['State_Num', 'Resident Population', 'Percent Change in Resident Population']


In [73]:
# create a dataset with no missing annual change values - just drop them

dfml = df[df["State_Num"].notna()]
dfml=dfml[~dfml.isin(["."]).any(axis=1)]

dfml = dfml[df["Resident Population"].notna()]
dfml=dfml[~dfml.isin(["."]).any(axis=1)]

dfml = dfml[df["Percent Change in Resident Population"].notna()]
dfml=dfml[~dfml.isin(["."]).any(axis=1)]

dfml = dfml[df["Annual Change (%)"].notna()]
dfml=dfml[~dfml.isin(["."]).any(axis=1)]
dfml["Annual Change(%)"] = dfml["Annual Change (%)"].astype(float)

/var/folders/hl/p3c4xh5x4nv00kvc7sckx1040000gn/T/ipykernel_72688/2545710660.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfml = dfml[df["Resident Population"].notna()]
/var/folders/hl/p3c4xh5x4nv00kvc7sckx1040000gn/T/ipykernel_72688/2545710660.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfml = dfml[df["Percent Change in Resident Population"].notna()]
/var/folders/hl/p3c4xh5x4nv00kvc7sckx1040000gn/T/ipykernel_72688/2545710660.py:12: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  dfml = dfml[df["Annual Change (%)"].notna()]


In [74]:
# define my target var
mltarget = dfml["Annual Change (%)"]
y_mltarget = mltarget.values

In [75]:
# define my features
mlfeatures = dfml[["Five-Digit ZIP Code", "year", "RECESSION_FLAG", "avg_rate_for_year", "Median_hh_income", "Year_Avg_Unempl", "State_Num", "Percent Change in Resident Population"]]
mlfeatures['year'] = pd.to_datetime(mlfeatures['year'], format='%Y')
mlfeatures['year'] = pd.to_datetime(mlfeatures['year']).dt.year
#mlfeatures.set_index('Year', inplace=True)
print(mlfeatures.dtypes)

Five-Digit ZIP Code                      float64
year                                       int64
RECESSION_FLAG                             int64
avg_rate_for_year                        float64
Median_hh_income                           int64
Year_Avg_Unempl                          float64
State_Num                                float64
Percent Change in Resident Population    float64
dtype: object


/var/folders/hl/p3c4xh5x4nv00kvc7sckx1040000gn/T/ipykernel_72688/833417539.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mlfeatures['year'] = pd.to_datetime(mlfeatures['year'], format='%Y')
/var/folders/hl/p3c4xh5x4nv00kvc7sckx1040000gn/T/ipykernel_72688/833417539.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mlfeatures['year'] = pd.to_datetime(mlfeatures['year']).dt.year


In [62]:
mlfeatures.head()

,Five-Digit ZIP Code,year,RECESSION_FLAG,avg_rate_for_year,Median_hh_income,Year_Avg_Unempl,State_Num,Percent Change in Resident Population
1,1001.0,1985,0,8.100833,56871,7.2,6.0,4.9
2,1001.0,1986,0,6.805000,58920,6.7,6.0,4.9
3,1001.0,1987,0,6.657500,59624,6.2,6.0,4.9
4,1001.0,1988,0,7.568333,60115,5.5,6.0,4.9
5,1001.0,1989,1,9.216667,61153,5.4,6.0,4.9


In [63]:
mlfeatures.shape

(414241, 8)

#### Creating training and testing datasets

In [76]:


# Split the data into training and testing sets
# First using random
mlfeatures_train, mlfeatures_test, mltarget_train, mltarget_test = train_test_split(mlfeatures, mltarget, test_size=0.2, random_state=42)
x_mlfeatures_train = mlfeatures_train.values #creating numpy versions here
x_mlfeatures_test = mlfeatures_test.values
y_mltarget_train = mltarget_train.values
y_mltarget_test = mltarget_test.values

# Print the shapes of the training and testing sets
print("Shape of mlfeatures_train:", mlfeatures_train.shape)
print("Shape of mltarget_train:", mltarget_train.shape)
print("Shape of mlfeatures_test:", mlfeatures_test.shape)
print("Shape of mltarget_test:", mltarget_test.shape)

# Second split: using dates
dfmltrim = dfml[["Five-Digit ZIP Code", "year", "RECESSION_FLAG", "avg_rate_for_year",'Median_hh_income','Year_Avg_Unempl', 'Annual Change (%)',"State_Num", "Percent Change in Resident Population"]]
train_date_df = dfmltrim[dfmltrim['year'] < 2019]
x_train_date_df = train_date_df.values
test_date_df = dfmltrim[dfmltrim['year'] >= 2019]
x_test_date_df = test_date_df.values
# Separate features and target variables
mlfeatures_train_date = train_date_df.drop('Annual Change (%)', axis=1)
x_mlfeatures_train_date = mlfeatures_train_date.values #create a numpy version 
mltarget_train_date = train_date_df['Annual Change (%)']
y_mltarget_train_date = mltarget_train_date.values # numpy again, and also below...
mlfeatures_test_date = test_date_df.drop('Annual Change (%)', axis=1)
x_mlfeatures_test_date = mlfeatures_test_date.values
mltarget_test_date = test_date_df['Annual Change (%)']
y_mltarget_test_date = mltarget_test_date.values

print("Shape of mlfeatures_train_date:", mlfeatures_train_date.shape)
print("Shape of mltarget_train_date:", mltarget_train_date.shape)
print("Shape of mlfeatures_test_date:", mlfeatures_test_date.shape)
print("Shape of mltarget_test_date:", mltarget_test_date.shape)


Shape of mlfeatures_train: (330850, 8)
Shape of mltarget_train: (330850,)
Shape of mlfeatures_test: (82713, 8)
Shape of mltarget_test: (82713,)
Shape of mlfeatures_train_date: (363775, 8)
Shape of mltarget_train_date: (363775,)
Shape of mlfeatures_test_date: (49788, 8)
Shape of mltarget_test_date: (49788,)


In [77]:
mlfeatures_train.head()

,Five-Digit ZIP Code,year,RECESSION_FLAG,avg_rate_for_year,Median_hh_income,Year_Avg_Unempl,State_Num,Percent Change in Resident Population
305803,18054.0,1996,0,5.298333,61225,5.5,2.0,3.4
101341,64110.0,2003,0,1.127500,63967,5.8,24.0,7.0
120391,78550.0,2021,0,0.080000,70784,5.1,28.0,15.9
308763,27504.0,2011,0,0.101667,60428,8.8,12.0,9.5
425576,76534.0,2020,1,0.375833,71186,5.1,28.0,15.9


In [78]:
mlfeatures_train.columns

Index(['Five-Digit ZIP Code', 'year', 'RECESSION_FLAG', 'avg_rate_for_year',
       'Median_hh_income', 'Year_Avg_Unempl', 'State_Num',
       'Percent Change in Resident Population'],
      dtype='object')

## Linear Regression

In [79]:
from sklearn.linear_model import LinearRegression

# Create a Linear Regression model for each
lr_model = LinearRegression()
lr_model_date = LinearRegression()

# Fit the model to the training data
lr_model.fit(x_mlfeatures_train, y_mltarget_train)
lr_model_date.fit(x_mlfeatures_train_date, y_mltarget_train_date)


LinearRegression()

In [80]:
from sklearn.metrics import mean_squared_error, r2_score

# Make predictions on the testing data
y_mltarget_pred = lr_model.predict(x_mlfeatures_test) # 80/20 split
y_mltarget_date_pred = lr_model_date.predict(x_mlfeatures_test_date) # date-based split

print("80-20 split results")
# Print the key stats: mse, rmse, r-squared
mse = mean_squared_error(y_mltarget_test, y_mltarget_pred)
print("Mean Squared Error:", mse)
rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)
r2 = r2_score(y_mltarget_test, y_mltarget_pred)
print("R-squared:", r2)


print("date-based results")
# Print the key stats: mse, rmse, r-squared
mse_date = mean_squared_error(y_mltarget_test, y_mltarget_pred)
print("Mean Squared Error:", mse_date)
rmse_date = np.sqrt(mse_date)
print("Root Mean Squared Error:", rmse_date)
r2_date = r2_score(y_mltarget_test_date, y_mltarget_date_pred)
print("R-squared:", r2)


80-20 split results
Mean Squared Error: 46.53990884890454
Root Mean Squared Error: 6.8220164796711344
R-squared: 0.22144269593885557
date-based results
Mean Squared Error: 46.53990884890454
Root Mean Squared Error: 6.8220164796711344
R-squared: 0.22144269593885557


In [81]:
# TESTING THE MODEL OUT


# Use the trained model to make predictions for the new dataset
predictions = lr_model.predict(new_data)
predictions_date = lr_model_date.predict(new_data)
# Print the predicted annual change (%) for each year
print("Predicted Annual Change (%):", predictions)
print("Predicted Annual Change (%):", predictions_date, " based on date-split train test")


ValueError: X has 9 features, but LinearRegression is expecting 8 features as input.

## Random Forest

In [82]:
from sklearn.ensemble import RandomForestRegressor

# Create a Random Forest model
rf_model = RandomForestRegressor(n_estimators=200, random_state=42)
# Fit the model to the training data
rf_model.fit(x_mlfeatures_train, y_mltarget_train)

y_mltarget_pred = rf_model.predict(x_mlfeatures_test)

In [92]:
print("Results based on 80-20 split")
# Print the key stats: mse, rmse, r-squared
mse = mean_squared_error(y_mltarget_test, y_mltarget_pred)
print("Mean Squared Error:", mse)
rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)
r2 = r2_score(y_mltarget_test, y_mltarget_pred)
print("R-squared:", r2)

Results based on 80-20 split
Mean Squared Error: 25.750482628421924
Root Mean Squared Error: 5.074493337114744
R-squared: 0.5692250623320806


In [85]:
# TESTING THE RANDOM SPLIT MODEL OUT


# Use the trained model to make predictions for the new dataset
predictions = rf_model.predict(new_data)

# Print the predicted annual change (%) for each year
print("Predicted Annual Change (%):", predictions)

Predicted Annual Change (%): [15.81585 15.81585 15.81585]


## Gradient Boost

In [86]:
from sklearn.ensemble import GradientBoostingRegressor

# Create an empty Gradient Boosting Regressor model
gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)

# Fit the model to the training data
gb_model.fit(x_mlfeatures_train, y_mltarget_train)

# Make predictions on the testing data
y_mltarget_pred = gb_model.predict(x_mlfeatures_test)


In [89]:

print("results from 80-20 split")
# Print the key stats: mse, rmse, r-squared
mse = mean_squared_error(y_mltarget_test, y_mltarget_pred)
print("Mean Squared Error:", mse)
rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)
r2 = r2_score(y_mltarget_test, y_mltarget_pred)
print("R-squared:", r2)


results from 80-20 split
Mean Squared Error: 25.750482628421924
Root Mean Squared Error: 5.074493337114744
R-squared: 0.5692250623320806


## Voting regressor to combine them

In [88]:
from sklearn.ensemble import VotingRegressor

# Create the voting regressor
voting_model = VotingRegressor([('rf', rf_model), ('gb', gb_model), ('lr', lr_model)])

# Fit the voting regressor to the training data
voting_model.fit(x_mlfeatures_train, y_mltarget_train)

# Make predictions on the testing data
y_mltarget_pred = voting_model.predict(mlfeatures_test)




/Users/galenmittermann/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but RandomForestRegressor was fitted without feature names
  warnings.warn(
/Users/galenmittermann/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but GradientBoostingRegressor was fitted without feature names
  warnings.warn(
/Users/galenmittermann/opt/anaconda3/lib/python3.9/site-packages/sklearn/base.py:443: UserWarning: X has feature names, but LinearRegression was fitted without feature names
  warnings.warn(


In [91]:
# Print the key stats: mse, rmse, r-squared
mse = mean_squared_error(y_mltarget_test, y_mltarget_pred)
print("Mean Squared Error:", mse)
rmse = np.sqrt(mse)
print("Root Mean Squared Error:", rmse)
r2 = r2_score(y_mltarget_test, y_mltarget_pred)
print("R-squared:", r2)

Mean Squared Error: 25.750482628421924
Root Mean Squared Error: 5.074493337114744
R-squared: 0.5692250623320806


In [93]:

# Use the trained model to make predictions for the new dataset
predictions = voting_model.predict(new_data)

# Print the predicted annual change (%) for each year
print("Predicted Annual Change (%):", predictions)

Predicted Annual Change (%): [11.76698263 11.75325561 11.73952858]
